In [1]:
%matplotlib inline
import os,random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle as cPickle, random, sys
import scipy
from sklearn import preprocessing
import tensorflow as tf
from numpy.fft import *


import tensorflow.keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
# import tensorflow as tf

In [2]:
Xd = cPickle.load(open("RML2016.10b.dat",'rb'), encoding='latin1')

In [3]:
INTENSITY_AXIS=0
QUADRATURE_AXIS=1

snrs,mods = map(lambda j: sorted(list(set(map(lambda x: x[j], Xd.keys())))), [1,0])


In [13]:
X_Intensity = []  
X_Quadrature = []
Label_strings = []
for mod in mods:
    X_Intensity.append(Xd[(mod,18)][:,INTENSITY_AXIS,:])
    X_Quadrature.append(Xd[(mod,18)][:,QUADRATURE_AXIS,:])
    for label in range(len(Xd[(mod,18)][:,INTENSITY_AXIS,:])):
        Label_strings.append(mod)
        
X_Intensity = np.vstack(X_Intensity) # Quick way to create stacked dataset
X_Quadrature = np.vstack(X_Quadrature)
Label_strings=np.vstack(Label_strings)
complex_num = X_Intensity + 1j*X_Quadrature

le= preprocessing.LabelEncoder()
le.fit(Label_strings)
Label= le.transform(Label_strings)
depth = 10
Label = tf.one_hot(Label,depth)


In [14]:
np.random.seed(2016)
X_Intensity_Dataset = np.c_[X_Intensity, Label]
X_Intensity_Dataset.shape

(60000, 138)

In [15]:
import copy
X_Intensity_Dataset_Label_shuffled = copy.copy(X_Intensity_Dataset)
np.random.shuffle(X_Intensity_Dataset_Label_shuffled)

In [16]:
length_of_X_train = int(0.7*X_Intensity_Dataset_Label_shuffled.shape[0]) #42000
length_of_X_validation = (int(0.2*X_Intensity_Dataset_Label_shuffled.shape[0]))+length_of_X_train #12000
print(length_of_X_validation)

54000


In [17]:
Train_Intensity_Dataset = X_Intensity_Dataset_Label_shuffled[0:length_of_X_train,0:128]
Y_Train_Intensity_Dataset = X_Intensity_Dataset_Label_shuffled[0:length_of_X_train,128:]

Validation_Intensity_Dataset = X_Intensity_Dataset_Label_shuffled[length_of_X_train:length_of_X_validation,0:128]
Y_Validation_Intensity_Dataset = X_Intensity_Dataset_Label_shuffled[length_of_X_train:length_of_X_validation,128:]

Validation_Intensity_Dataset.shape


(12000, 128)

In [18]:
# Y_train.shape
BATCH_SIZE = 100

In [24]:

Train_Intensity_Dataset=Train_Intensity_Dataset.reshape(42000,128,1)
Validation_Intensity_Dataset=Validation_Intensity_Dataset.reshape(12000,128,1)
print(Validation_Intensity_Dataset.shape)

(12000, 128, 1)


In [25]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

model = Sequential()
model.add(LSTM(128, input_shape=((128,1)), activation='relu', return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(64, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(10, activation='softmax'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 128, 128)          66560     
_________________________________________________________________
dropout_9 (Dropout)          (None, 128, 128)          0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dropout_10 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_11 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)               

In [26]:
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)

model.compile(
    loss='categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)
#print(Train_Dataset.shape)
model.fit(Train_Intensity_Dataset,
          Y_Train_Intensity_Dataset,
          batch_size=BATCH_SIZE,
          epochs=100)

Epoch 1/100
323/420 [======================>.......] - ETA: 7s - loss: 2.3028 - accuracy: 0.0998

KeyboardInterrupt: 

In [ ]:
from tensorflow.keras.layers import Reshape,Dense,Dropout,Activation,Flatten
from tensorflow.keras.layers import GlobalMaxPooling1D,Conv1D,MaxPooling1D
model_m = Sequential()
model_m.add(Conv1D(100, 10, activation='relu', input_shape=(128,1)))
model_m.add(Conv1D(100, 10, activation='relu'))
model_m.add(MaxPooling1D(3))
model_m.add(Conv1D(160, 10, activation='relu'))
model_m.add(Conv1D(160, 10, activation='relu'))
model_m.add(GlobalMaxPooling1D())
model_m.add(Dropout(0.5))
model_m.add(Dense(10, activation='softmax'))
print(model_m.summary())

In [ ]:
model_m.compile(loss='categorical_crossentropy',
                optimizer='adam', metrics=['accuracy'])

In [ ]:
EPOCHS = 300
history = model_m.fit(Train_Intensity_Dataset,
                      Y_Train_Intensity_Dataset,
                      batch_size=BATCH_SIZE,
                      epochs=EPOCHS,
                     validation_data=(Validation_Intensity_Dataset, Y_Validation_Intensity_Dataset),
                    callbacks = [tensorflow.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')])

In [ ]:
_, accuracy = model_m.evaluate(Validation_Intensity_Dataset, Y_Validation_Intensity_Dataset)
print('Accuracy: %.2f' % (accuracy*100))

In [ ]:
plt.figure()
plt.title('Training performance')
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='Validation')
plt.legend()
plt.show()

